# Laboratorio de regresión - 5

|                |   |
:----------------|---|
| **Nombre**     | André Esteban Vera  |
| **Fecha**      | 18/09/2025 |
| **Expediente** | 745232  |

## Validación

Hemos estado usando `train_test_split` en nuestros modelos anteriores.

¿Por qué?

Para separar nuestros datos en dos partes y hacerles una regresión por separado y comparar los resultados

Si la muestra es un subset de la población y queremos generalizar sobre la población, ¿no sería mejor utilizar todos los datos al entrenar un modelo?

El propósito de volver a muestrear dentro de nuestro dataset es tener una idea de qué tan buena podría ser la generalización de nuestro modelo. Imagina un dataset ya separado en dos mitades. Utilizas la primera mitad para entrenar el modelo y pruebas en la segunda mitad; la segunda mitad eran datos invisibles para el modelo al momento de entrenar. Esto nos lleva a tres escenario típicos:

1. Si el modelo hace buenas predicciones en la segunda mitad, significa que la primera mitad era "suficiente" para generalizar.
2. Si el modelo no hace buenas predicciones en la segunda mitad, pero sí en la primera mitad, podría ser que había información importante en la segunda mitad que debió haber sido tomada en cuenta al entrenar, o un problema de overfitting.
3. Si el modelo no hace buenas predicciones en la segunda mitad, y tampoco en la primera mitad, se tendrían que revisar los factores y/o el modelo seleccionado.

El caso ideal sería el 1, pero por estadística los errores y varianzas tienen como entrada el número de muestas, por lo que tenemos menos seguridad de nuestros resutados al usar menos muestras. Si vemos que el modelo generaliza bien podemos unir de nuevo el dataset y entrenar sobre el dataset completo.

En el caso 2 está el problema de que no podemos saber qué información es necesaria para el entrenamiento apropiado del modelo; esto nos lleva a pensar que debemos usar el dataset completo para entrenar, pero esto nos lleva al mismo problema de no saber si el modelo puede generalizar.

El problema sólo incrementa si se tienen hiperparámetros en el modelo (e.g. $\lambda$ en regularización).

## Leave-One-Out Cross Validation

Este método de validación es una colección de $n$ `train-test-split`. Teniendo un dataset de $n$ muestras, la lógica es:
1. Saca una muestra del dataset.
2. Entrena tu modelo con las $n-1$ muestras.
3. Evalúa tu modelo en la muestra que quedó fuera con el métrico que más se ajuste a la aplicación.
4. Regresa la muestra al dataset.
5. Repite 1-4 con muestras diferentes hasta haber hecho el procedimiento $n$ veces para $n$ muestras.
6. Calcula la media y desviación estándar de los métricos guardados.

Con los resultados del proceso de validación podemos saber qué tan bueno podría ser el modelo seleccionado con los datos (con/sin transformaciones).

### Ejercicio 1

Utiliza el dataset `Motor Trend Car Road Tests`. Elimina la columna `model` y entrena 32 modelos diferentes utilizando Leave-One-Out Cross Validation con target `mpg`. Utiliza MSE como métrico.

In [18]:
import pandas as pd
datos = pd.read_excel('Motor Trend Car Road Tests.xlsx')
datos = datos.drop(columns = 'model')
datos.head()

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


In [19]:
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import numpy as np

mses = []

y = datos['mpg']
x = datos.drop(columns = 'mpg')
n = len(x)

for i in range(n):
    X_train = x.drop(i)
    y_train = y.drop(i)
    X_test = x.iloc[[i]]
    y_test = y.iloc[[i]]

    model = LinearRegression()
    model.fit(X_train, y_train)

    y_hat = model.predict(X_test)

    mse = mean_squared_error(y_test, y_hat)
    mses.append(mse)


media_mses = np.mean(mses)
desv_mses = np.std(mses)

print(f'Media MSE: {media_mses}')
print(f'Desviacion estandar MSE: {desv_mses}')

Media MSE: 12.181558006901954
Desviacion estandar MSE: 17.067399871888544


Interpreta.

## K-Folds Cross-Validation

El dataset `Motor Trend Car Road Tests` sólo tiene 32 muestras, y utilizar un modelo sencillo de regresión múltiple hace que usar LOOCV sea muy rápido. El dataset `California Housing` tiene $20640$ muestras para $9$ columnas, entonces realizar un ajuste sobre una transformación o sobre el modelo y luego calcular el impacto esperado podría tomar más tiempo.

La solución propuesta es dividir el dataset en *k* folds (partes iguales), ajustar en *k-1* folds y probar en el restante.

### Ejercicio 2
Utiliza el dataset `California Housing` y haz K-folds Cross Validation con 10 folds. Utiliza el MSE como métrico.

In [20]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
print("Dataset Shape:", housing.data.shape, housing.target.shape)
print("Dataset Features:", housing.feature_names)
print("Dataset Target:", housing.target_names)
X = housing.data
y = housing.target

Dataset Shape: (20640, 8) (20640,)
Dataset Features: ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
Dataset Target: ['MedHouseVal']


In [21]:
# K-folds CROSS VALIDATION - Shuffle
from sklearn.utils import shuffle
shuffle?

Signature: shuffle(*arrays, random_state=None, n_samples=None)
Docstring:
Shuffle arrays or sparse matrices in a consistent way.

This is a convenience alias to ``resample(*arrays, replace=False)`` to do
random permutations of the collections.

Parameters
----------
*arrays : sequence of indexable data-structures
    Indexable data-structures can be arrays, lists, dataframes or scipy
    sparse matrices with consistent first dimension.

random_state : int, RandomState instance or None, default=None
    Determines random number generation for shuffling
    the data.
    Pass an int for reproducible results across multiple function calls.
    See :term:`Glossary <random_state>`.

n_samples : int, default=None
    Number of samples to generate. If left to None this is
    automatically set to the first dimension of the arrays.  It should
    not be larger than the length of arrays.

Returns
-------
shuffled_arrays : sequence of indexable data-structures
    Sequence of shuffled copies of 

In [ ]:
import numpy as np
X_shuffled, y_shuffled = shuffle(X, y, random_state=69)

k_folds = 10
n_samples = X_shuffled.shape[0]
fold_size = n_samples // k_folds

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

mse_scores = []

for fold in range(10):
    # Definir el inicio y fin del fold
    inicio = fold * fold_size
    fin = inicio + fold_size if fold < 9 else n_samples  # El último fold toma el resto
    
    # Seleccionar datos de prueba y entrenamiento
    X_test = X_shuffled[inicio:fin]
    y_test = y_shuffled[inicio:fin]
    X_train = np.concatenate([X_shuffled[:inicio], X_shuffled[fin:]], axis=0)
    y_train = np.concatenate([y_shuffled[:inicio], y_shuffled[fin:]], axis=0)
    
    modelo = LinearRegression()
    modelo.fit(X_train, y_train)
    
    y_pred = modelo.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mse_scores.append(mse)

print("MSE promedio:", np.mean(mse_scores))
print("Desviación estándar del MSE:", np.std(mse_scores))



MSE promedio: 0.5291483318403645
Desviación estándar del MSE: 0.0288070204250697


Interpreta.

El MSE promedio de 0.53 nos dice que el modelo se puede llegar a equivocar prediciendo de una manera algo común 
Y desviación estándar baja significa que el modelo generaliza bien y es estable.

## Referencia

James, G., Witten, D., Hastie, T., Tibshirani, R.,, Taylor, J. (2023). An Introduction to Statistical Learning with Applications in Python. Cham: Springer. ISBN: 978-3-031-38746-3